# Import libraries

In [2]:
import pandas as pd
import sqlite3

In [3]:
filepath = 'D:\\Sadaf SBU\\master\\term 3\\Advance Data Mining\\database.sqlite'
cnx=sqlite3.connect(filepath)

# Databases

### Station

In [4]:
Station = pd.read_sql_query("SELECT * FROM Station",con=cnx)
Station.head()

,id,name,lat,long,dock_count,city,installation_date
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013


### Trip (The database we use in this exercise)

In [5]:
Trip = pd.read_sql_query("SELECT * FROM Trip",con=cnx)
Trip.head()

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
0,4069,174,8/29/2013 9:08,2nd at South Park,64,8/29/2013 9:11,2nd at South Park,64,288,Subscriber,94114
1,4073,1067,8/29/2013 9:24,South Van Ness at Market,66,8/29/2013 9:42,San Francisco Caltrain 2 (330 Townsend),69,321,Subscriber,94703
2,4074,1131,8/29/2013 9:24,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,317,Subscriber,94115
3,4075,1117,8/29/2013 9:24,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,316,Subscriber,94122
4,4076,1118,8/29/2013 9:25,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,322,Subscriber,94597


### Status

The volume of the Status database was too large and could not be loaded

### Weather

In [30]:
Weather = pd.read_sql_query("SELECT * FROM Weather",con=cnx)
Weather.head()

,date,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f,mean_dew_point_f,min_dew_point_f,max_humidity,mean_humidity,min_humidity,...,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,zip_code
0,8/29/2013,74,68,61,61,58,56,93,75,57,...,10,10,23,11,28,0,4,,286,94107
1,8/30/2013,78,69,60,61,58,56,90,70,50,...,10,7,29,13,35,0,2,,291,94107
2,8/31/2013,71,64,57,57,56,54,93,75,57,...,10,10,26,15,31,0,4,,284,94107
3,9/1/2013,74,66,58,60,56,53,87,68,49,...,10,10,25,13,29,0,4,,284,94107
4,9/2/2013,75,69,62,61,60,58,93,77,61,...,10,6,23,12,30,0,6,,277,94107


## 2.1) Retrieve the ids of the bikes who have rented bikes for more than 10 days in total.

In [6]:
q ="""SELECT bike_id, duration
FROM Trip 
WHERE duration >= 864000; 
"""
#864000 = 24*60*60*10

df =pd.read_sql_query(q,con=cnx)
df

,bike_id,duration
0,535,17270400
1,680,1852590
2,466,2137000
3,262,1133540


## 2.2) Display the top 5 bikes that have been rented the most frequently, along with the count of their rentals.



In [27]:
q ="""SELECT bike_id, COUNT(*) AS rental_count
FROM Trip
GROUP BY bike_id
ORDER BY rental_count DESC
LIMIT 5
"""
df =pd.read_sql_query(q,con=cnx)
df

,bike_id,rental_count
0,392,2061
1,489,1975
2,558,1955
3,267,1951
4,631,1948


## 2.3) Retrieve the start and end station names for trips taken by users who have rented bikes for a duration of more than 1 hour.

In [7]:
q ="""SELECT start_station_name, end_station_name
FROM Trip
WHERE duration > 3600
"""
df =pd.read_sql_query(q,con=cnx)
df

,start_station_name,end_station_name
0,Redwood City Caltrain Station,Redwood City Caltrain Station
1,South Van Ness at Market,San Francisco Caltrain 2 (330 Townsend)
2,Spear at Folsom,Spear at Folsom
3,Embarcadero at Vallejo,Market at Sansome
4,Embarcadero at Vallejo,Market at Sansome
...,...,...
21237,Embarcadero at Vallejo,Golden Gate at Polk
21238,Embarcadero at Sansome,Embarcadero at Sansome
21239,Embarcadero at Vallejo,Embarcadero at Sansome
21240,Embarcadero at Sansome,Embarcadero at Sansome


## 2.5) Calculate the percentage of trips that were subscribed by customers (not casual users) for each season, considering only those trips that lasted longer than 15 minutes.

In [9]:
q ="""WITH filtered_trips AS (
    SELECT
        subscription_type,
        SUBSTR(start_date, 1, 1) AS month
    FROM
        trip
    WHERE
        duration > 900
)
SELECT
    CASE
        WHEN month IN ('12', '1', '2') THEN 'Winter'
        WHEN month IN ('3', '4', '5') THEN 'Spring'
        WHEN month IN ('6', '7', '8') THEN 'Summer'
        WHEN month IN ('9', '10', '11') THEN 'Fall'
        ELSE 'Unknown'
    END AS season,
    (COUNT(CASE WHEN subscription_type = 'Customer' THEN 1 END) * 100.0) / COUNT(*) AS percentage_subscribed
FROM
    filtered_trips
GROUP BY
    season;
"""
df =pd.read_sql_query(q,con=cnx)
df

,season,percentage_subscribed
0,Fall,65.107071
1,Spring,54.815843
2,Summer,56.395753
3,Winter,52.579239


## 2.6) Find the most popular station by calculating the ratio of the average number of trips per station to the average number of trips for all stations.

In [10]:
q ="""SELECT start_station_name, COUNT(*) AS num_of_trips_per_station
FROM trip
GROUP BY start_station_name
ORDER BY num_of_trips_per_station DESC
"""
df =pd.read_sql_query(q,con=cnx)
df

,start_station_name,num_of_trips_per_station
0,San Francisco Caltrain (Townsend at 4th),49092
1,San Francisco Caltrain 2 (330 Townsend),33742
2,Harry Bridges Plaza (Ferry Building),32934
3,Embarcadero at Sansome,27713
4,Temporary Transbay Terminal (Howard at Beale),26089
...,...,...
69,San Mateo County Center,287
70,Franklin at Maple,224
71,Redwood City Public Library,213
72,Broadway at Main,67
